In [1]:
import warnings
warnings.simplefilter(action='ignore')
import nltk
import pandas as pd

In [2]:
train_data = pd.read_csv(r"D:\dev0608\treino.csv", delimiter=';')
test_data = pd.read_csv(r"D:\dev0608\avaliacao.csv", delimiter=';')

In [3]:
train_data

,cod_treino,titulo,categoria,cor,tamanho,marca,Caracteristica,tipo,referencia,voltagem,ambiente,subcategoria,linha,imagem
0,1,"Piso Laminado Click Nature 136x21,7cm m² Artens",Piso Laminado,Nature,"136x21,7",Eucafloor,Nature,Pisos,NaN,NaN,Interno,Pisos,Click,NaN
1,2,"Piso Laminado Click Elmo Ravena 136x21,7cm m² ...",Piso Laminado,Ravena,"136x21,7",Eucafloor,Elmo Ravena,Pisos,NaN,NaN,Interno,Pisos,Click,NaN
2,3,"Piso Laminado Click Legno Blanche 136x21,7cm m...",Piso Laminado,Blanche,"136x21,7",Eucafloor,Legno Blanche,Pisos,NaN,NaN,Interno,Pisos,Click,NaN
3,4,"Piso Laminado Click Evidence Kalahari 136x21,7...",Piso Laminado,Bege,"136x21,7",Eucafloor,Evidence Kalahari,Pisos,NaN,NaN,Interno,Pisos,Click,NaN
4,5,"Piso Laminado Click Spot Carvalho 134x18,7cm m...",Piso Laminado,Carvalho,"134x18,7",Eucafloor,Spot Carvalho,Pisos,NaN,NaN,Interno,Pisos,Click,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,376,Mesa Bistrô Wooden Mel e Preta,Mesa,Mel & Preta,NaN,NaN,NaN,Moveis para cozinha,NaN,NaN,Cozinha,Moveis para Cozinha e Sala de estar,NaN,NaN
376,377,Cadeira Base em Madeira e Aço Eames Preto,Cadeira,Preto,NaN,NaN,NaN,Madeira e Aço,NaN,NaN,Cozinha,Moveis para Cozinha e Sala de estar,NaN,NaN
377,378,Kit 2 Cadeiras Madeira Maciça Tiê Cabecasa Mad...,Cadeira,Amêndoa & Bege claro,NaN,NaN,Madeira Maciça,Madeira e Aço,NaN,NaN,Cozinha,Moveis para Cozinha e Sala de estar,NaN,NaN
378,379,Rack Para TV até 60 Polegadas 3 Portas 217cm N...,Rack,Branco,NaN,NaN,TV até 60 Polegadas,Madeira,Rack Para TV,NaN,Sala & Escritorio,Moveis para Sala,NaN,NaN


In [4]:
train_data.columns

Index(['cod_treino', 'titulo', 'categoria', 'cor', 'tamanho', 'marca',
       'Caracteristica', 'tipo', 'referencia', 'voltagem', 'ambiente',
       'subcategoria', 'linha', 'imagem'],
      dtype='object')

In [5]:
import pandas as pd
import spacy
import unidecode
import re
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from spacy.lang.pt.stop_words import STOP_WORDS

nlp = spacy.load("pt_core_news_lg")

def preprocess_text(text: str) -> str:
    text = text.lower()
    text = unidecode.unidecode(text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = ' '.join(text.split())
    doc = nlp(text)
    filtered_tokens = [token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in STOP_WORDS]
    processed_text = " ".join(filtered_tokens)
    return processed_text

def create_embeddings(texts):
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
    model = BertModel.from_pretrained('bert-base-multilingual-cased')
    
    embeddings = []
    
    for text in texts:
        input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
        with torch.no_grad():
            output = model(input_ids)
        
        embeddings.append(output.last_hidden_state.mean(dim=1).numpy().flatten())  # Flattening the 2D array
    
    return np.array(embeddings)

def classify_product(query_text, treino_info, keywords_mapping, ambiente_mapping, tipo_mapping, subcategoria_mapping):
    query_text = preprocess_text(query_text)
    for keyword, category in keywords_mapping.items():
        if keyword in query_text:
            return category

    for keyword, ambiente in ambiente_mapping.items():
        if keyword in query_text:
            return ambiente

    for keyword, tipo in tipo_mapping.items():
        if keyword in query_text:
            return tipo
    
    for keyword, subcategoria in subcategoria_mapping.items():
        if keyword in query_text:
            return subcategoria

    return None

def find_most_similar_product(query_embedding, treino_embeddings, treino_info):
    similarity_scores = cosine_similarity(query_embedding.reshape(1, -1), treino_embeddings)
    most_similar_index = np.argmax(similarity_scores)
    return treino_info.loc[most_similar_index]

def main():
    treino = pd.read_csv("treino.csv", sep=";")
    avaliacao = pd.read_csv("avaliacao.csv", sep=";")
    
    treino['titulo_processed'] = treino['titulo'].apply(preprocess_text)
    avaliacao['titulo_processed'] = avaliacao['titulo'].apply(preprocess_text)
    
    treino_embeddings = create_embeddings(treino['titulo_processed'])
    avaliacao_embeddings = create_embeddings(avaliacao['titulo_processed'])
    
    treino_info = treino[['categoria', 'ambiente', 'tipo', 'subcategoria']]
    
    keywords_mapping = {
        'cuba': 'Cuba',
        'sofa': 'Sofas', 
        'rack': 'Rack',
        'cadeira': 'Cadeira',
        'mesa': 'Mesa',
        'piso': 'Piso Vinilico',
        'torneira': 'Torneiras',
        'guarda-roupa': 'Guarda-Roupa',
        'espelho': 'Espelhos',
        'tapete': 'Tapetes',
        'rodape': 'Rodapés'
    }
    
    ambiente_mapping = {
        'cozinha': 'Cozinha',
        'sala': 'Sala & Escritorio',
        'quarto': 'Quarto',
        'banheiro': 'Banheiro',
        'interno': 'Interno'
    }

    tipo_mapping = {
        'piso': 'Pisos',
        'cuba': 'Torneiras',
        'madeira': 'Madeira e Aço',
        'inox': 'Cuba Inox',
        'sofa': 'Sofas',
        'rack': 'Rack',
        'guarda-roupa': 'Guarda-Roupa',
        'tapete': 'Tapetes',
        'espelho': 'Espelhos',
        'mesa': 'Mesa',
        'rodape': 'Rodapés'
    }
    
    subcategoria_mapping = {
        'espelho': 'Espelhos',
        'piso': 'Pisos',
        'cadeira': 'Cadeiras',
        'sofa': 'Sofas'
       
    }
    
    avaliacao['categoria'] = ''
    avaliacao['ambiente'] = ''
    avaliacao['tipo'] = ''
    avaliacao['subcategoria'] = ''
    
    for i in range(len(avaliacao)):
        query_text = avaliacao.loc[i, 'titulo']
        categoria_classificada = classify_product(query_text, treino_info, keywords_mapping, ambiente_mapping, tipo_mapping, subcategoria_mapping)
        
        if categoria_classificada:
            avaliacao.at[i, 'categoria'] = categoria_classificada
            if categoria_classificada == 'Guarda-Roupa':
                avaliacao.at[i, 'ambiente'] = 'Quarto'
            if categoria_classificada == 'Sofas':
                avaliacao.at[i, 'tipo'] = 'Sofas'
        else:
            most_similar_product = find_most_similar_product(avaliacao_embeddings[i], treino_embeddings, treino_info)
            avaliacao.at[i, 'categoria'] = most_similar_product['categoria']
            avaliacao.at[i, 'ambiente'] = most_similar_product['ambiente'] if most_similar_product['ambiente'] != '' else 'Nao encontrado'
            avaliacao.at[i, 'tipo'] = most_similar_product['tipo'] if most_similar_product['tipo'] != '' else 'Nao encontrado'
            avaliacao.at[i, 'subcategoria'] = most_similar_product['subcategoria'] if most_similar_product['subcategoria'] != '' else 'Nao encontrado'
    
    print(avaliacao[['titulo', 'categoria']])

if __name__ == "__main__":
    main()





Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
                                               titulo            categoria
0   Piso Vinílico em Manta Residence Jura Brown Ma...        Piso Vinilico
1   Piso Vinílico Cola Ambienta Rústico Cinnamon m...        Piso Vinilico
2   Piso Vinílico Cola Injoy Gerbera Madeira Bege ...        Piso Vinilico
3   Rodapé Cerâmica Brilhante RHDS1000 15X61cm Artens              Rodapés
4   Cuba para Cozinha Dupla de Embutir 82,5x17cm A...                 Cuba
5   Cuba Inox Pia Cozinha Gourmet Embutir Polido G...                 Cuba
6  